# Preprocessing of MIMIC-CXR dataset

This notebook illustrates how week supervision can be applied on X-rays and radiology reports.  
[MIMIC-CXR](https://physionet.org/content/mimic-cxr/2.0.0/) Database is a large publicly available dataset of chest X-rays including radiology reports. It contains 377110 images and 227835 radiographic studies.
[MIMIC-CXR-JPG](https://physionet.org/content/mimic-cxr-jpg/2.0.0/) Database also includes weak labels which are derived from the radiology reports using CheXpert labler. 


## Getting all set up

To get access to the data, one needs to be a "credential user" in PhysioNet and sign the data use agreement. 
1. To get a credentialed PhysioNet account, follow the following [instructions](https://physionet.org/about/citi-course/). 
2. Then sign the data use agreement for both, the MIMIC-CXR and the MIMIC-CXR-JPG database. 
3. Download the files
    "cxr-record-list.csv.gz" (from mimic-cxr),
    "cxr-study-list.csv.gz" (from mimic-cxr),
    "mimic-cxr-2.0.0-split.csv.gz" (from mimic-cxr-jpg),
    "mimic-cxr-2.0.0-chexpert.csv.gz" (from mimic-cxr-jpg)
    and unzip them with 7zip   
4. set your working directory to this folder

In [ ]:
#os.chdir("")

## Imports

In [17]:
import os
import numpy as np
import pandas as pd
import random
import copy
import torch.optim as optim
import csv
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torchvision.models as models
import itertools
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from typing import Dict
from joblib import dump, load

## Downloading MIMIC-CXR-JPG images

Firstly, we are going to download the first part of our dataset: X-rays images. For that, you will need credentialed PhysioNet account. Before executing the following please:
1. check if you have enough memory space in the current environment (the first 1000 images are 1.51GB large)
2. check if wget is installed on your device
3. set n
4. insert your physionet username and password

In [2]:
#set n between 1 and 377110
n = 1000
record_list = pd.read_csv("cxr-record-list.csv").to_numpy()
study_list = pd.read_csv("cxr-study-list.csv").to_numpy()

In [ ]:
username = "your_username_here"
password = "your_pw_here"

#image download - run only once
for i in tqdm(range(n)):
    url = ["wget -r -N -c -np --user=", username, " --password=", password, " https://physionet.org/files/mimic-cxr-jpg/2.0.0/",record_list[i,3]]
    command = "".join(url)
    command = "".join([command.replace(".dcm", ""),".jpg"])
    os.system(command)


## Downloading MIMIC-CXR reports

Next, we download the reports. 

In [ ]:
#report download - run only once
url = ["wget -r -N -c -np --user=", username, " --password=", password, " https://physionet.org/files/mimic-cxr/2.0.0/mimic-cxr-reports.zip"]
command = "".join(url)
os.system(command)

Unzip the mimic-cxr-reports folder using 7zip.

Since the reports are all stored in separate files, we process them such that we get one csv where each report corresponds to one row. We extract the "Findings" and "Impressions" sections from each txt file and save it to one csv. 

In [ ]:
with open('mimic_cxr_text.csv', 'w', newline='', encoding='utf-8') as f:
    for i in tqdm(range(len(study_list))):
        with open(''.join(["mimic-cxr-reports/", study_list[i,2]])) as f_path:
            text = ''.join(f_path.readlines())
        text = text.replace("\n", "")
        text = text.replace(",", "")
        start = text.find("FINDINGS:")
        end = text.find("IMPRESSION:")
        findings = text[start:end]
        impressions = text[end:len(text)]
        row = [study_list[i,0],study_list[i,1], findings, impressions]
        csvwriter = csv.writer(f)
        csvwriter.writerow(row)

In [3]:
#open
reports = pd.read_csv("mimic_cxr_text.csv", names = ["subject_id","study_id", "findings", "impressions"])

In the section "Findings" contains a description of the important aspects in the image and the section "Impression" gives a short summary of the most immediately relevant findings. For our rules, we use the "Impression" section, or if impressions are not available the "Finding" sections. About 5% of the reports are formated differently(do not have impression or findings sections), these we ignore in this analysis. 

In [4]:
#missing values are denoted by "." in impressions -> nan
loc = np.where(reports['impressions'] == '.')[0]
reports['impressions'][loc] = np.nan
print("number of NAs in findings and impressions:", pd.isna(reports[['findings', 'impressions']]).sum())

#if impression is missing insert finding
reports.impressions.fillna(reports.findings, inplace=True)
#if non of both are there, we do not analyse this study
del reports['findings']
reports_processed = reports.dropna()

number of NAs in findings and impressions: findings       78174
impressions    39692
dtype: int64


<ipython-input-4-524708893a8e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports['impressions'][loc] = np.nan
C:\Users\marli\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Some reports refer to multiple images. Now we merge the reports to our record list such that each image gets a report assigned if a report is available. 

In [5]:
#merge reports to record_list
record_list = pd.read_csv("cxr-record-list.csv")
record_report_list = pd.merge(record_list, reports_processed, how = 'left', on= ['study_id','subject_id'])

## Adding labels and split
Now we want to build an array, which contains all needed information of the images. The "cxr_record_list.csv" contains the columns (`subject_id`,`study_id`,`dicom_id`) which all together form a unique ID and `path`. Now we want to merge the labels and the split information to each image.  

But before we can do that, we need to specify the labels:

### Labels

The CheXpert labler detect the presence of 14 diagnoses in radiology reports. 
"mimic-cxr-2.0.0-chexpert.csv" contains one column for each of the 14 diagnoses and one row for each study. Each study and each diagnosis has assigned one of the four values: 1.0,-1.0, 0.0 or missing. They have the following interpretation:

* 1.0 The label was positively mentioned in the associated study, and is present in one or more of the corresponding images
* 0.0 The label was negatively mentioned in the associated study, and therefore should not be present in any of the corresponding images
* -1.0 The label was either: (1) mentioned with uncertainty in the report, and therefore may or may not be present to some degree in the corresponding image, or (2) mentioned with ambiguous language in the report and it is unclear if the pathology exists or not
* Missing (empty element) - No mention of the label was made in the report

So we are primarily interested in the 1.0s.
One study can have multiple labels positively mentioned, like it is the case in row 7 below.

In [6]:
split_list = pd.read_csv("mimic-cxr-2.0.0-split.csv")
labels_chexpert = pd.read_csv("mimic-cxr-2.0.0-chexpert.csv")

labels_chexpert.iloc[:,2:].head(8) #note that we removed the ids in the output due to data privacy

,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,1.0,NaN,1.0,NaN,NaN


To get one label for each study, we do the following:
If there is exactly one label positively mentioned, this label will be assigned to the study. If there are multiple labels positively mentioned, one label will be chosen randomly. When there is no 1.0 assigned to the study, the label will be set to 'No Finding'.

In [7]:
#initialise labels with 0
labels_chexpert['label'] = 0
labels_list = labels_chexpert.columns.to_numpy()
#iterate through labels: 
#three cases: only one, non, or multiple diagnoses
for i in tqdm(range(len(labels_chexpert))):
    #which labels are 1? 
    label_is1 = labels_chexpert.iloc[i,:] == 1.0
    if (sum(label_is1)==1):
       labels_chexpert.iloc[i,16] = labels_list[label_is1]
    elif sum(label_is1) > 1:
        labels_chexpert.iloc[i,16] = random.choice(labels_list[label_is1])
    else: 
        labels_chexpert.iloc[i,16] = 'No Finding'

100%|██████████| 227827/227827 [10:35<00:00, 358.71it/s]


In [8]:
labels_chexpert['label'].head(8)

0          No Finding
1          No Finding
2          No Finding
3          No Finding
4       Consolidation
5          No Finding
6          No Finding
7    Pleural Effusion
Name: label, dtype: object

In [9]:
#change names to ids
labels = {'Atelectasis':0,
               'Cardiomegaly':1,
               'Consolidation':2,
               'Edema':3,
               'Enlarged Cardiomediastinum':4,
               'Fracture':5,
               'Lung Lesion':6,
               'Lung Opacity':7,
               'Pleural Effusion':8,
               'Pneumonia':9,
               'Pneumothorax':10,
               'Pleural Other':11,
               'Support Devices':12,
               'No Finding':13}        
        
for i in tqdm(range(len(labels_chexpert))):
 labels_chexpert.iloc[i,16] = labels.get(labels_chexpert.iloc[i,16])

100%|██████████| 227827/227827 [08:40<00:00, 437.50it/s]


### Merge labels and splits

Next, the records, reports, labels and split are merged to one file. Note that sometimes one report refers to multiple X-rays (e.g. one frontal and one lateral X-ray) and since the labels are derived from the reports, one label might refer to multiple images.

In [11]:
#merge records, labels and split
record_report_split_list = pd.merge(record_report_list, split_list, how = 'left', on = ['dicom_id', 'study_id','subject_id'])
record_report_split_label_list = pd.merge(record_report_split_list, labels_chexpert.iloc[:,[0,1,16]], how = 'left', on = ['study_id','subject_id'])

The train/validation/test split is taken from mimic-cxr-jpg, which is intended to provide a reference dataset split for studies using this data. 

In [12]:
print("train-val-test split proportions:" ,record_report_split_label_list.groupby('split').size()/len(record_report_split_label_list))

train-val-test split proportions: split
test        0.013680
train       0.978388
validate    0.007931
dtype: float64


In [13]:
print("train-val-test split proportions:" ,record_report_split_label_list.groupby('split').size()/len(record_report_split_label_list))

train-val-test split proportions: split
test        0.013680
train       0.978388
validate    0.007931
dtype: float64


There are 14 classes, where about 42% of the studies are assigned to the class 'No finding'. We should keep in mind that we are dealing with an unbalanced dataset.

Now the labels get replaced by their id. This gives us the final "input_list". Since we will need this file multiple times and the runtime is relatively long, it is cached.

In [14]:
input_list_full = record_report_split_label_list

In [15]:
#save the whole file
dump(input_list_full, "input_list.lib")

['input_list.lib']

We will continue working with the input list of the first n rows where the rows including missing values are dropped. Instead of n=1000 we now have n=932

In [19]:
#open only first n rows
input_list_pd = load("input_list.lib").iloc[:n,:]
#drop nas
input_list = input_list_pd.dropna().to_numpy()
#save new n
n = len(input_list)
print(n)

932


## Creating rules from Chexpert-labler

We are using the ["mentions" from Chexpert-labler](https://github.com/stanfordmlgroup/chexpert-labeler/tree/master/phrases/mention) for building our rules. Therefore, we need to download the .txt files corresponding to each class from github. The file names are the labels we defined above, where all letters are lowercase and instead of whitespaces there are underscores. So first we do some transformations to get a list of our classes. 

In [20]:
#download synonym list from chexpert
classes = list(labels)
#lower case
classes = [each_string.lower() for each_string in classes]
#replace whitespace with _
classes = [each_string.replace(" ", "_") for each_string in classes]
labels2ids = {classes[i]:i for i in range(14)}

Now we create a new folder in our directory and store the txt files there. 

In [ ]:
#create folder chexpert_rules
os.makedirs("".join([os.getcwd(),"/chexpert_rules"]))
#store files in folder
for i in range(len(classes)):
    os.system("".join(["curl https://raw.githubusercontent.com/stanfordmlgroup/chexpert-labeler/master/phrases/mention/", 
                       classes[i], ".txt ", "-o chexpert_rules/", classes[i], ".txt"]))

### T Matrix
The T matrix contains information about which rule corresponds to which label. In the following snippets, we build this matrix. 

In [21]:
#read txt in
lines = {}
for i in range(len(classes)):
    with open("".join(["chexpert_rules/", classes[i], ".txt"])) as f:
        lines[classes[i]] = [each_string.replace("\n", "") for each_string in f.readlines()]
          
mentions = pd.DataFrame({'label': label, 'rule': rule} for (label, rule) in lines.items())
mentions.head()

,label,rule
0,atelectasis,"[atelecta, collapse]"
1,cardiomegaly,"[cardiomegaly, the heart, heart size, cardiac ..."
2,consolidation,[consolidat]
3,edema,"[edema, heart failure, chf, vascular congestio..."
4,enlarged_cardiomediastinum,"[_mediastinum, cardiomediastinum, contour, med..."


In [22]:
#building the dataframe "rules"
rules = pd.DataFrame([i for i in itertools.chain.from_iterable(mentions['rule'])], columns = ["rule"])
rules['rule_id'] = range(len(rules))
rules['label'] = np.concatenate([np.repeat(mentions['label'][i], len(mentions['rule'][i])) for i in range(14)])
rules['label_id'] = [labels2ids[rules['label'][i]] for i in range(len(rules))]
rules.head()

,rule,rule_id,label,label_id
0,atelecta,0,atelectasis,0
1,collapse,1,atelectasis,0
2,cardiomegaly,2,cardiomegaly,1
3,the heart,3,cardiomegaly,1
4,heart size,4,cardiomegaly,1


In [23]:
rule2rule_id = dict(zip(rules["rule"], rules["rule_id"]))
rule2label = dict(zip(rules["rule_id"], rules["label_id"]))

def get_mapping_rules_labels_t(rule2label: Dict, num_classes: int) -> np.ndarray:
    """ Function calculates t matrix (rules x labels) using the known correspondence of relations to decision rules """
    mapping_rules_labels_t = np.zeros([len(rule2label), num_classes])
    for rule, labels in rule2label.items():
        mapping_rules_labels_t[rule, labels] = 1
    return mapping_rules_labels_t

mapping_rules_labels_t = get_mapping_rules_labels_t(rule2label, len(labels2ids))
print("T matrix:", mapping_rules_labels_t[0:5,:])
print("shape:", mapping_rules_labels_t.shape)

T matrix: [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
shape: (148, 14)


We derive our T matrix with 148 rules for 14 classes. 

Now we want to check if there are any rules which assign to the same class. Indeed, there is one case: rule "defib" builds a rule for two different classes. In our T matrix, "defib" occures in two rows, since knodle cannot handle one rule assigned to multiple classes. 

In [24]:
print(len(np.unique(rules['rule'])) == len(rules['rule']))
rules_size = rules.groupby('rule').size()
print(rules_size[np.where(rules_size > 1)[0]])
#rule defib appears for two different classes

False
rule
defib    2
dtype: int64


### Z Matrix
The Z matrix contains the information wheter a rule matches to an observation or not. Its shape is #instances x #rules. 

In [26]:
def get_rule_matches_z (data: np.ndarray, num_rules: int) -> np.ndarray:
    """
    Function calculates the z matrix (samples x rules)
    data: np.array (reports)
    output: sparse z matrix
    """
    rule_matches_z = np.zeros([len(data), num_rules])
    for ind in range(len(data)):
        for rule, rule_id in rule2rule_id.items():
            if rule in (data[ind]):
                rule_matches_z[ind, rule_id] = 1
    return rule_matches_z

rule_matches_z = get_rule_matches_z(input_list[:,4], (len(rule2rule_id)+1))

dump(rule_matches_z, "rule_matches_z.lib")

['rule_matches_z.lib']

# Image encoding 
For the image encoding we try two different approaches:
1. extract features using a pretrained model (without finetuning)
2. finetune a pretrained model and then extract features

## Extracting features using a pretrained model (without finetuning)

For the implementation we use pytorch and the pretrained resnet50 from torchvision. 
The following class loads the data and transforms it in the way it is required for resnet50. It is written in the form such that it is compatible with torch.utils.data.DataLoader. 

In [27]:
class mimicDataset(Dataset):
    
    def __init__(self, path):
        'Initialization'
        self.path = path
        #self.y = y
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.path)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        image = Image.open("".join(["physionet.org/files/mimic-cxr-jpg/2.0.0/",self.path[index,3].replace(".dcm", ".jpg")])).convert('RGB')
        X = self.transform(image)
        label = self.path[index,6]
        
        return X, torch.tensor(label)
    
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

We load the resnet50 model remove the last layer, so the output is then the second last layer with dimension 1x2048.


In [28]:
model = models.resnet50(pretrained=True)
modules = list(model.children())[:-1]
model=torch.nn.Sequential(*modules)
for p in model.parameters():
    p.requires_grad = False
    
model.eval()
#apply modified resnet50 to data
dataloaders = DataLoader(mimicDataset(input_list[:n,:]), batch_size=n,num_workers=0)
    
data, labels = next(iter(dataloaders))
with torch.no_grad():
    features_var = model(data)
    features = features_var.data 
    all_X = features.reshape(n,2048).numpy()
    
#save feature matrix
dump(all_X, "all_X.lib")

['all_X.lib']

## Finetuning a pretrained CNN and extracting the second last layer as features

In the second approach of image encoding, we use the concept of transfer learning. 
Therefore, we take a pretrained CNN and continue training the model with our data. This saves a lot of time and leads to satisfying results even on a small dataset. 
For the implementation we again use pytorch and the pretrained resnet50 from torchvision. 

First, we split the data according to the split given in mimic-cxr-jpg. 

In [29]:
input_train = input_list[input_list[:,5] == 'train',:]
input_validate = input_list[input_list[:,5] == 'validate',:]
input_test = input_list[input_list[:,5] == 'test',:]

Since the dataset is unbalanced, we use a weighted sampler 

In [30]:
class_counts = np.zeros(14)
for i in range(14): class_counts[i] = sum(input_train[:,6]==i)
weight = 1/class_counts
sample_weights = np.array([weight[t] for t in input_train[:,6]])
sample_weights = torch.from_numpy(sample_weights)
sample_weights = sample_weights.double()
sampler = torch.utils.data.WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights))

In [31]:
dataset = {'train' : mimicDataset(input_train),
           'val': mimicDataset(input_validate),
           'test':  mimicDataset(input_test)}

dataloaders = {'train': DataLoader(dataset['train'] , batch_size=4, num_workers=0, sampler = sampler),
               'val': DataLoader(dataset['val'] , batch_size=4, num_workers=0 )}

dataset_sizes = {x: len(dataset[x]) for x in ['train', 'val']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

The following function whcih trains the model is taken from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [32]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if phase == 'val':
                    print('predictions',preds)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print('Best val Acc: {:4f}'.format(best_acc), )

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


Now we initialize the model:
We change the last layer to 14, since we only have 14 classes. We choose the Cross Entropy Loss as an optimization cirterion, the adam optimizer and an adaptive learning rate. 
The model is finetuned using only two epochs, since more epochs would overfit the model. 

In [33]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
# set output size to 14 (number of classes)
model.fc = nn.Linear(num_ftrs, 14)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)

Epoch 0/1
----------
train Loss: 1.8165 Acc: 0.4488
predictions tensor([ 8,  8,  1, 13])
predictions tensor([8, 8, 1, 1])
predictions tensor([13,  8, 13,  1])
predictions tensor([13, 12])
val Loss: 2.7613 Acc: 0.0714

Epoch 1/1
----------
train Loss: 1.1641 Acc: 0.6416
predictions tensor([3, 0, 5, 9])
predictions tensor([0, 0, 0, 0])
predictions tensor([ 9,  0,  9, 13])
predictions tensor([13, 13])
val Loss: 2.8807 Acc: 0.0714

Best val Acc: 0.071429


Now, that we have trained our model, we will extract the features. 

In the next step, the last layer is removed from the model, so the output is then the second last layer with dimension 1x2048.
Then we let all out data run through the model. 

In [34]:
modules = list(model.children())[:-1]
model=torch.nn.Sequential(*modules)
for p in model.parameters():
    p.requires_grad = False
    
model.eval()
#apply modified resnet50 to data
dataloaders = DataLoader(mimicDataset(input_list[:n,:]), batch_size=n,num_workers=0)
    
data, labels = next(iter(dataloaders))
with torch.no_grad():
    features_var = model(data)
    features = features_var.data 
    all_X_finetuned = features.reshape(n,2048).numpy()


In [35]:
#save feature matrix
dump(all_X_finetuned, "all_X_finetuned.lib")

['all_X_finetuned.lib']